# Leakage Coefficient Summary

> *This notebook summarize the leakage coefficient fitted from 4 dsDNA samples.*

## Import software

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'  # for hi-dpi displays

In [ ]:
figure_size = (5, 4)
default_figure = lambda: plt.subplots(figsize=figure_size)
save_figures = True

def savefig(filename, **kwargs):
    if not save_figures:
        return
    import os
    dir_ = 'figures/'
    kwargs_ = dict(dpi=300, bbox_inches='tight')
                   #frameon=True, facecolor='white', transparent=False)
    kwargs_.update(kwargs)
    plt.savefig(dir_ + filename, **kwargs_)

## Data files

In [ ]:
bsearch_str = 'DexDem'
leakage_kde = pd.read_csv(
    'results/Multi-spot - leakage coefficient all values KDE %s.csv' % bsearch_str, index_col=0)
leakage_gauss = pd.read_csv(
    'results/Multi-spot - leakage coefficient all values gauss %s.csv' % bsearch_str, index_col=0)
nbursts = pd.read_csv(
    'results/Multi-spot - leakage coefficient all values nbursts %s.csv' % bsearch_str, index_col=0)
for df in (leakage_kde, leakage_gauss, nbursts):
    df.columns.name = 'Channel'

In [ ]:
for dx in (leakage_gauss, leakage_kde, nbursts):
    dx.columns = pd.Index(np.arange(1, 9), name='Spot')

In [ ]:
PLOT_DIR = './figure/'

## Plot style

In [ ]:
bmap = sns.color_palette("Set1", 9)
colors_dark = np.array(bmap)[(1,0,2,3,4,8), :]
colors_dark4 = np.array(bmap)[(1,0,2,8), :]

bmap = sns.color_palette('Paired', 12)
colors_light = np.array(bmap)[(0,4,2,8,6,10), :]
colors_light4 = np.array(bmap)[(0,4,2,8), :]
colors_light[-1] = colors_light4[-1] = [.8, .8, .8]

colors_paired = np.zeros((colors_dark.shape[0]*2, colors_dark.shape[1]))
colors_paired[::2] = colors_dark
colors_paired[1::2] = colors_light
colors_paired4 = colors_paired[(0, 1, 2, 3, 4, 5, 10, 11), :]
sns.palplot(colors_paired4)

In [ ]:
sns.set(style='ticks', font_scale=1.4, palette=colors_paired)

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()

kws = dict(lw=2, marker='o', ms=8)
for i, did in enumerate(('7d', '12d', '17d', 'DO')):
    (100*leakage_kde).loc[did].plot(label='%s KDE' % did, ax=ax, color=colors_light4[i], **kws)
    nbursts.loc[did].plot(ax=ax2, ls='--', lw=2.5, color=colors_dark4[i])

for i, did in enumerate(('7d', '12d', '17d', 'DO')):    
    (100*leakage_gauss).loc[did].plot(label='%s Gauss' % did, ax=ax, color=colors_dark4[i], **kws)
    
handles, lab = ax.get_legend_handles_labels()
h = handles#[1::2] + handles[::2]
l = lab[1::2] + lab[::2]
ax.legend(ncol=2, loc=1, bbox_to_anchor=(1, 0.5), borderaxespad=0.)
ax.set_ylim(0)

ax2.set_ylim(0, 3200)
plt.xlim(0.75, 8.25)
plt.xlabel('Channel')
ax.set_ylabel('Leakage %')
ax2.set_ylabel('# Bursts')
sns.despine(offset=10, trim=True, right=False)
savefig('multi-spot leakage KDE vs Gauss.svg')

In [ ]:
# bmap = sns.color_palette("Set1", 9)
# colors = np.array(bmap)[(1,0,2,3,4,8,6,7), :]
# sns.set_palette(colors)
# sns.palplot(colors)

In [ ]:
sns.swarmplot(data=leakage_kde);
plt.figure()
sns.swarmplot(data=leakage_kde.T);

# Average leakage

### Mean per sample:

In [ ]:
lk_s = pd.DataFrame(index=['mean', 'std'], columns=leakage_kde.index, dtype=float)

lk_s.loc['mean'] = leakage_kde.mean(1)*100
lk_s.loc['std'] = leakage_kde.std(1)*100
lk_s = lk_s.round(5)
lk_s

### Mean per sample (weighted on the number of bursts):

Number of bursts in D-only population:

In [ ]:
nbursts

In [ ]:
leakage_kde

In [ ]:
lk_sw = pd.DataFrame(index=['mean', 'std'], columns=leakage_kde.index, dtype=float)

lk_sw.loc['mean'] = (nbursts*leakage_kde).sum(1)/nbursts.sum(1)*100
lk_sw.loc['std'] = np.sqrt((((leakage_kde.T*100 - lk_sw.loc['mean']).T**2) * nbursts).sum(1) / (nbursts.sum(1) - 1))
#lk_sw['mean'] = (nbursts * lk_sw).sum(1) / nbursts.sum(1).sum()
lk_sw = lk_sw.round(5)
lk_sw

In [ ]:
lk_swg = pd.DataFrame(index=['mean', 'std'], columns=leakage_gauss.index, dtype=float)

lk_swg.loc['mean'] = (nbursts*leakage_gauss).sum(1)/nbursts.sum(1)*100
lk_swg.loc['std'] = np.sqrt((((leakage_gauss.T*100 - lk_swg.loc['mean']).T**2) * nbursts).sum(1) / (nbursts.sum(1) - 1))
#lk_sw['mean'] = (nbursts * lk_sw).sum(1) / nbursts.sum(1).sum()
lk_swg = lk_swg.round(5)
lk_swg

In [ ]:
lk_sw_m = pd.concat((lk_sw.loc['mean'], lk_swg.loc['mean']), axis=1, keys=['KDE', 'Gauss'])
lk_sw_m

In [ ]:
lk_sw_s = pd.concat((lk_sw.loc['std'], lk_swg.loc['std']), axis=1, keys=['KDE', 'Gauss'])
lk_sw_s

In [ ]:
sns.set_style('ticks')

In [ ]:
lk_sw_m.plot(yerr=lk_sw_s, lw=5, alpha=0.6)
plt.xlim(-0.2, 3.2)
plt.xticks(range(4), lk_sw_s.index)
sns.despine(trim=True, offset=10)

In [ ]:
lk_sw_m.plot.bar(yerr=lk_sw_s, alpha=0.8)
sns.despine(trim=True, offset=10)

In [ ]:
sns.swarmplot(data=leakage_kde*100, size=8, palette=colors_dark);
plt.ylim(0)
lk_sw_m.loc[:,'KDE'].plot(lw=3, alpha=0.8, color='k')
plt.xlim(-0.2, 3.2)
plt.xticks(range(4), lk_sw_s.index)
sns.despine(trim=True, offset=10)

### Mean per channel:

In [ ]:
lk_c = pd.DataFrame(index=['mean', 'std'], columns=leakage_kde.columns, dtype=float)

lk_c.loc['mean'] = leakage_kde.mean()*100
lk_c.loc['std'] = leakage_kde.std()*100
#lk_c['mean'] = lk_c.mean(1)
lk_c = lk_c.round(5)
lk_c

### Mean per channel (weighted on the number of bursts):

In [ ]:
lk_cw = pd.DataFrame(index=['mean', 'std'], columns=leakage_kde.columns, dtype=float)

lk_cw.loc['mean'] = (nbursts*leakage_kde).sum()/nbursts.sum()*100
lk_cw.loc['std'] = np.sqrt((((leakage_kde*100 - lk_cw.loc['mean'])**2) * nbursts).sum(0) / (nbursts.sum(0) - 1))
#lk_cw['mean'] = lk_cw.mean(1)
lk_cw = lk_cw.round(5)
lk_cw

In [ ]:
lk_cwg = pd.DataFrame(index=['mean', 'std'], columns=leakage_gauss.columns)

lk_cwg.loc['mean'] = (nbursts*leakage_gauss).sum()/nbursts.sum()*100
lk_cwg.loc['std'] = np.sqrt((((leakage_kde*100 - lk_cwg.loc['mean'])**2) * nbursts).sum(0) / (nbursts.sum(0) - 1))
#lk_cwg['mean'] = lk_cwg.mean(1)
lk_cwg = lk_cwg.round(5)
lk_cwg

In [ ]:
lk_cw_m = pd.concat((lk_cw.loc['mean'], lk_cwg.loc['mean']), axis=1, keys=['KDE', 'Gauss'])
lk_cw_m.T

In [ ]:
lk_cw_s = pd.concat((lk_cw.loc['std'], lk_cwg.loc['std']), axis=1, keys=['KDE', 'Gauss'])
lk_cw_s.T

In [ ]:
sns.set_palette(colors_dark)

In [ ]:
kws = dict(lw=5, marker='o', ms=8, alpha=0.5)
lk_cw.loc['mean'].plot(yerr=lk_cw.loc['std'], **kws)
lk_cwg.ix['mean',:].plot(yerr=lk_cwg.loc['std'],**kws)
plt.ylim(0, 4)
plt.xlim(0.75, 8.25)
sns.despine(trim=True)

In [ ]:
lk_cw_m.plot.bar(alpha=0.8)
#sns.despine(trim=True, offset=10)

Transform table in tidy form:

In [ ]:
leakage_kde_t = pd.melt(leakage_kde.reset_index(), id_vars=['Sample'], 
                        value_name='leakage_kde').apply(pd.to_numeric, errors='ignore')
leakage_kde_t.leakage_kde *= 100
leakage_kde_t.head()

In [ ]:
_ = lk_cw_m.copy().assign(Spot=range(1, 9)).set_index('Spot')
_.head()

In [ ]:
sns.set_palette(colors_dark4)

In [ ]:
sns.swarmplot(x='Spot', y='leakage_kde', data=leakage_kde_t, size=6, hue='Sample');
_ = lk_cw_m.copy().assign(Spot=range(8)).set_index('Spot')
_.loc[:,'KDE'].plot(lw=3, alpha=0.8, color='k')
plt.ylim(0)
plt.xlim(-0.25, 7.25)
sns.despine(trim=True)

> **NOTE:** There is a per-channel trend that cannot be ascribed to the background 
> because we performend a D-emission burst search and selection and the leakage vs ch
> does not resemble the D-background vs channel curve.
>
> The effect is probably due to slight PDE variations (detectors + optics) that 
> slightly change $\gamma$ on a per-spot basis.

### Weighted mean of the weighted mean

In [ ]:
leakage_kde_wmean = (leakage_kde*nbursts).sum().sum() / nbursts.sum().sum()
leakage_kde_wmean

# Figure

In [ ]:
%config InlineBackend.figure_format='retina'  # for hi-dpi displays

Now I will transform `leakage_kde` in "tidy form" for
easier plotting.

For info on "data tidying" see:

- http://stackoverflow.com/questions/37490771/seaborn-categorical-plot-with-hue-from-dataframe-rows/
- https://www.ibm.com/developerworks/community/blogs/jfp/entry/Tidy_Data_In_Python

In [ ]:
leakage_kde

In [ ]:
leakage_kde_t = pd.melt((100*leakage_kde).reset_index(), id_vars=['Sample'], 
                        value_name='leakage_kde').apply(pd.to_numeric, errors='ignore')
leakage_kde_t.head()

In [ ]:
# leakage_kde_t = pd.melt((100*leakage_kde).T.reset_index(), id_vars=['Spot'], 
#                         value_name='leakage_kde').apply(pd.to_numeric, errors='ignore')
# leakage_kde_t.head()

In [ ]:
sns.set_palette(colors_dark4)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
plt.subplots_adjust(wspace=0.1)

sns.swarmplot(x='Sample', y='leakage_kde', data=leakage_kde_t, size=6, ax=ax[0])
lk_sw_m.loc[:,'KDE'].plot(lw=3, alpha=0.8, color='k', ax=ax[0])
ax[0].set_ylim(0)
ax[0].set_xlim(-0.2, 3.2)
plt.xticks(range(4), lk_sw_s.index)
sns.despine(trim=True, offset=10, ax=ax[0])

sns.swarmplot(x='Spot', y='leakage_kde', data=leakage_kde_t, size=6, hue='Sample', ax=ax[1])
_ = lk_cw_m.copy().assign(Spot=range(8)).set_index('Spot')
_.loc[:,'KDE'].plot(lw=3, alpha=0.8, color='k', label='mean')
ax[1].set_ylim(0)
ax[1].set_xlim(-0.25, 7.25)
plt.xticks(np.arange(8));
sns.despine(trim=True, offset=10, ax=ax[1], left=True)
ax[1].yaxis.set_visible(False)
ax[0].set_ylabel('Leakage %')
leg = ax[1].get_legend()
h, l = ax[1].get_legend_handles_labels()
ax[1].legend(h[1:] + h[:1], l[1:] + l[:1], title='Sample', loc='lower right')
fs = 28
ax[0].text(0,0, 'A', fontsize=fs)
ax[1].text(0,0, 'B', fontsize=fs)
savefig('multi-spot leakage KDE 2panels.png')

# Save

## Per-channel mean

In [ ]:
lk_cw.to_csv('results/Multi-spot - leakage coefficient mean per-ch KDE %s.csv' % bsearch_str)

## Per-sample mean

In [ ]:
lk_sw.to_csv('results/Multi-spot - leakage coefficient mean per-sample KDE %s.csv' % bsearch_str)

## Global mean

In [ ]:
'%.5f' % leakage_kde_wmean

In [ ]:
with open('results/Multi-spot - leakage coefficient KDE wmean %s.csv' % bsearch_str, 'w') as f:
    f.write('%.5f' % leakage_kde_wmean)